In [4]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from tensorflow.keras.activations import linear, relu, sigmoid
from modules.utils import change_X_data_1, change_y_data, change_X_data_2, create_y_test_file



In [7]:
X = pd.read_csv("/Users/elouan/Data/Data_ENSxCFM/X_train.csv")
y = pd.read_csv("/Users/elouan/Data/Data_ENSxCFM/y_train.csv")

In [23]:
X.head()

,obs_id,venue,order_id,action,side,price,bid,ask,bid_size,ask_size,trade,flux
0,0,4,0,0,1,3.0,0.0,1.0,4.615121,0.693147,0,100
1,0,4,1,0,0,-1.7,0.0,1.0,4.615121,0.693147,0,100
2,0,4,2,1,1,2.8,0.0,1.0,4.615121,0.693147,0,-100
3,0,4,3,0,1,3.0,0.0,1.0,4.615121,0.693147,0,100
4,0,4,4,1,1,3.7,0.0,1.0,4.615121,0.693147,0,-100


In [8]:
X2 = change_X_data_2(X)


/Users/elouan/anaconda3/envs/challenge_ENS_env_CFM/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [22]:
X2.head()

,column 1
0,"[[4.0, 0.0, 0.0, 1.0, 3.0, 0.0, 1.0, 4.6151204..."
1,"[[4.0, 0.0, 0.0, 0.0, -0.8, 0.0, 4.0, 1.098612..."
2,"[[4.0, 0.0, 0.0, 1.0, 0.1, 0.0, 1.0, 6.947937,..."
3,"[[4.0, 0.0, 0.0, 1.0, 2.1, 0.0, 20.0, 6.527958..."
4,"[[4.0, 0.0, 1.0, 1.0, 0.1, 0.0, 1.0, 6.302619,..."


In [ ]:
# Access value in row 2 (index), column 'column_name'
value = X2.at[2, 'column 1']  # or df.loc[2, 'column_name']
print(value)


In [ ]:
X2.info()

In [9]:
X3 = np.array([np.array(row) for row in X2["column 1"]])

In [ ]:
X3[0][1]

In [ ]:
X3.shape

In [10]:
#y = change_y_data(y)
y1 = y.drop(columns=["obs_id"])

y1.head()

,eqt_code_cat
0,10
1,15
2,0
3,13
4,0


In [ ]:
y1.info()

In [11]:
y2 = np.array(y1["eqt_code_cat"])
y2.shape

(160800,)

In [12]:
print(np.any(np.isnan(X3)))  # Check features for NaNs
print(np.any(np.isnan(y2)))

True
False


In [13]:
X3 = np.nan_to_num(X3)

In [14]:
dataset = tf.data.Dataset.from_tensor_slices((X3, y2))

In [ ]:
for x, y in dataset.take(3):  # Use `.take()` to limit output to 3 samples
    print("Features:", x.numpy())
    print("Label:", y.numpy())


In [ ]:
print(dataset.element_spec)


In [15]:
batch_size = 32
dataset = dataset.shuffle(buffer_size=len(X3))  # Shuffling the dataset



In [16]:
dataset = dataset.batch(batch_size)

In [17]:
for x, y in dataset.take(1):
    print(f"Features batch shape: {x.shape}, Labels batch shape: {y.shape}")

Features batch shape: (32, 100, 11), Labels batch shape: (32,)


2024-11-12 14:51:35.340291: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [   GRU(64, return_sequences=True, input_shape = (100, 11) ),            
        Dropout(0.2),
        GRU(64),
        Dropout(0.2),
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(24, activation = 'softmax',   name = "output")
        
        
    ], name = "first_multiclass_model" 
)

In [21]:
model.summary()

Model: "first_multiclass_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_8 (GRU)                     │ (None, 100, 64)        │        14,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 100, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_9 (GRU)                     │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L1 (Dense)                      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 24)             │           792 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 127,850 (499.42 KB)

 Trainable params: 42,616 (166.47 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 85,234 (332.95 KB)

In [18]:
model = tf.keras.models.load_model("../models/model_matrix_3.h5")

In [19]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

history = model.fit(
    dataset,
    epochs=50,
)

Epoch 1/50


5025/5025 ━━━━━━━━━━━━━━━━━━━━ 362s 71ms/step - accuracy: 0.6773 - loss: 0.9361
Epoch 2/50
5025/5025 ━━━━━━━━━━━━━━━━━━━━ 360s 71ms/step - accuracy: 0.6766 - loss: 0.9391
Epoch 3/50
5025/5025 ━━━━━━━━━━━━━━━━━━━━ 344s 68ms/step - accuracy: 0.6811 - loss: 0.9285
Epoch 4/50
5025/5025 ━━━━━━━━━━━━━━━━━━━━ 340s 67ms/step - accuracy: 0.6799 - loss: 0.9306
Epoch 5/50
5025/5025 ━━━━━━━━━━━━━━━━━━━━ 366s 73ms/step - accuracy: 0.6807 - loss: 0.9291
Epoch 6/50
5025/5025 ━━━━━━━━━━━━━━━━━━━━ 372s 74ms/step - accuracy: 0.6778 - loss: 0.9310
Epoch 7/50
5025/5025 ━━━━━━━━━━━━━━━━━━━━ 382s 76ms/step - accuracy: 0.6800 - loss: 0.9300
Epoch 8/50
5025/5025 ━━━━━━━━━━━━━━━━━━━━ 404s 80ms/step - accuracy: 0.6797 - loss: 0.9285
Epoch 9/50
5025/5025 ━━━━━━━━━━━━━━━━━━━━ 395s 78ms/step - accuracy: 0.6815 - loss: 0.9279
Epoch 10/50
5025/5025 ━━━━━━━━━━━━━━━━━━━━ 388s 77ms/step - accuracy: 0.6812 - loss: 0.9261
Epoch 11/50
5025/5025 ━━━━━━━━━━━━━━━━━━━━ 395s 78ms/step - accuracy: 0.6811 - loss: 0.9242
Epoch 12

In [20]:
model.save("../models/model_matrix_4.h5")  # Saves the model as an HDF5 file


In [ ]:
# test_loss, test_accuracy = model.evaluate(dataset)
# print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
# # Reshape it to (1, 100, 11) to match the model input shape
# single_sequence = np.expand_dims(x_test_3[0], axis=0)

# # Make a prediction with the trained model
# prediction = model.predict(single_sequence)
# yhat = np.argmax(prediction)

# print("Prediction for the single sequence:", yhat)


In [ ]:
# n = 14

# prediction = model.predict(np.array(x_test_3.iloc[n]).reshape(1,12))
# prediction_p = tf.nn.softmax(prediction)
# yhat = np.argmax(prediction_p)
# print(prediction_p)
# print("prediction :")
# print(yhat)
# print("expected value : ")
# print(y.iloc[n])

In [ ]:
# m = 1000
# S = 0

# for i in range(m):
    
#     prediction = model.predict(np.array(x_test.iloc[i]).reshape(1,12))
#     prediction_p = tf.nn.softmax(prediction)
#     yhat = np.argmax(prediction_p)

#     if yhat == y["eqt_code_cat"].iloc[i] :
#         S+=1
    
# print(f"{(S/m)*100}% of precision")

    